In [37]:
import numpy as np
import pandas as pd
import jieba
import jieba.analyse
import jpype
import collections
import re
import matplotlib.pyplot as plt
from pylab import rcParams
from zhon.hanzi import punctuation
from pyhanlp import PerceptronLexicalAnalyzer

%matplotlib inline

In [2]:
def get_more_key_word_by_ner(content):
    analyer = PerceptronLexicalAnalyzer()
    s = str(analyer.analyze(content))
    s = combine_word_in_bucket(s)
    analyze_list = s.split(" ")
    # fetch ner j+n
    jn_list = fetch_ner_jn(analyze_list)
    # filter noun
    analyze_list = list(filter(lambda x: bool(re.search("/nr|/ns|/nt|/nz|/nx|/j", x)), analyze_list))
    analyze_list += jn_list
    return list(set(analyze_list))

In [189]:
def make_key_word_res(analyze_list):
    res_dict = {"nr": [], "ns": [], "nt": [], "nz": [], "nx": [], "j":[], "j+n":[]}
    # 去除非英文數字中文
    rule = re.compile(r"[^a-zA-Z0-9\u4e00-\u9fa5]")
    for analyze_text in analyze_list:
        nourn = analyze_text.split("/")[0]
        ner_type = analyze_text.split("/")[1]
        if not bool(rule.search(nourn)) and res_dict.get(ner_type) is not None:
            res_dict[ner_type].append(nourn)
    return res_dict

In [4]:
def combine_word_in_bucket(s):
    combine_word_list = re.findall("\[.*?\]", s)
    for combine_word in combine_word_list:
        result = "".join(re.findall("[\u4e00-\u9fa5]+", combine_word))
        s = s.replace(combine_word, result)
    return s

In [5]:
def fetch_ner_jn(analyze_list):
    jn_list = []
    for i in range(len(analyze_list)-1):
        if analyze_list[i][-1:] == "j" and analyze_list[i+1][-1:] == "n":
            jn = analyze_list[i].split("/")[0] + analyze_list[i+1].split("/")[0]
            jn_list.append(jn+"/j+n")
    return jn_list

In [6]:
def remove_punctuation(text, strip_all=True):
    if strip_all:
        rule = re.compile(r"[^a-zA-Z0-9\u4e00-\u9fa5]")
        text = rule.sub("",text)
    else:
        re_punctuation = "[{}]+".format(punctuation)
        text = re.sub(re_punctuation, "", text)
    return text.strip().replace("\n","")

In [38]:
def TraditionalChinese2SimplifiedChinese(sentence_str):
    HanLP = jpype.JClass('com.hankcs.hanlp.HanLP')
    return HanLP.convertToSimplifiedChinese(sentence_str)

In [118]:
def SimplifiedChinese2TraditionalChinese(sentence_str):
    HanLP = jpype.JClass('com.hankcs.hanlp.HanLP')
    return HanLP.convertToTraditionalChinese(sentence_str)

In [139]:
def perform_hanlp_key_word(text):
    HanLP = jpype.JClass("com.hankcs.hanlp.HanLP")
    text = TraditionalChinese2SimplifiedChinese(text)
    s = str(HanLP.extractKeyword(text, 15))
    s = SimplifiedChinese2TraditionalChinese(s)
    keyword_list = s.replace(" ", "")[1:-1].split(',')
    return keyword_list

In [107]:
def perform_hanlp_summary(text):
    HanLP = jpype.JClass("com.hankcs.hanlp.HanLP")
    keyword_list = str(HanLP.extractSummary(text, 15)).replace(" ", "")[1:-1].split(',')
    return keyword_list

In [8]:
# 載入不同主題資料
topic_list = ["政治", "科技", "娛樂", "體育", "社會", "財經", "健康", "國際"]
with open("data/text/big_data/corpus/" + topic_list[6] + ".txt", "r", encoding="utf-8") as content:
    content_list = [line.strip().replace(' ', '') for line in content]

In [185]:
content_list[4]

'▲男子看診卻意外爆出「愛滋三角戀」連醫師都看傻。（示意圖／Pixabay）記者李佳蓉／採訪報導診間狀況百百種，醫生除了得面對患者們的疑難雜症，竟連如八點檔的劇情也親身經歷！整形外科醫師朱育瑩在《醫師好辣》節目上分享一名40多歲男子因外傷住進加護病房，帶了五顏六色的藥物，自述是C型肝炎的抗病毒藥物，醫師覺得怪異並請男子太太將藥單帶來，沒想到經過藥物辨識後竟意外爆出三角戀，彷彿八點檔的「誇張劇情」神展開！狀況百出的醫院，沒想到也能爆出三角戀？整形外科醫師朱育瑩接受《ETtoday健康雲》採訪時表示，一名40多歲的男性患者因工傷住進加護病房，自述因染C肝一天需服用4顆五顏六色的藥物，藥物數量讓醫師感到奇怪，為了確保用藥安全，她請男子太太將藥單帶到醫院，沒想到在藥物辨識下發現全部都是「抗愛滋病」的藥物。朱醫師說：「知道這消息，護理人員心頭一震，因為該病人送急診時鮮血淋漓，擔心與他接觸過的醫護人員剛好因針扎而接觸到血液」。醫師再三與男子確認「你知道這是什麼藥物嗎？」但他堅持是抗C肝病毒的，直到醫師說出是「抗愛滋病毒藥物」才戳破他謊言。原來該男患者已染愛滋病多年，且檢查發現他的愛滋病病毒量高，顯然過去都未按時服藥。▼朱育瑩醫師在節目上分享連他都傻眼的案例。（圖／翻攝自《醫師好辣》Youtube）某天太太來到病房，發現丈夫使用後的醫療廢棄物處理方式與其他患者不同，房間垃圾桶都有不一樣的隔離，開始起疑，就連醫師都難以對她說出「妳老公是愛滋病患者」的殘忍事實。直到太太翻出先生的處方箋，依循藥名上網一個個查，得知自己也是受害者的她淚流滿面，哭訴自己被隱瞞多年。她向閨蜜哭訴，沒想到意外爆出閨蜜就是老公小三的真相。朱醫師說，最後正宮與小三一起到醫院驗愛滋病抗體，但讓人意外的是，該名男子的父母早知兒子染有愛滋病，且過去曾有過一段婚姻，還生下2個小孩，前妻也是因被他傳染愛滋病而死，留下的2名孩子都有垂直性的感染，誇張的劇情讓醫師看了也搖頭。▼不僅正宮及小三受害，男子多年前的婚姻也因愛滋病而毀。（圖／達志／示意圖）提到愛滋病，朱醫師解釋，愛滋病全名為後天免疫缺乏症候群（AcquiredImmuneDeficiencySyndrome，簡稱AIDS），其會造成免疫系統的破壞，主要靠體液及血液傳輸。然而大多數人以為只有「同志」才會染病，她解釋，在這世上大多數的愛滋病患者都是異性戀，主要因同性戀者並

In [140]:
perform_hanlp_key_word(content_list[1022])

['醫療',
 '醫院',
 '南投',
 '緊急',
 '救護',
 '副院長',
 '洪',
 '世昌',
 '服務',
 '醫師',
 '民國',
 '醫學',
 '土石流',
 '擔任',
 '急診']

In [111]:
perform_hanlp_summary(content_list[27])

['（圖／營養師簡鈺樺提供）營養師簡鈺樺先就這次推出的黑巧克力慕絲蛋糕做營養成分說明',
 '總重100公克、熱量342.2大卡、蛋白質6.6公克、脂肪23.4公克、碳水化合物26.3公克',
 '5.熱量消耗要吃就要動',
 '可以發現有將近6成熱量都是脂肪',
 '將近6成熱量都是脂肪',
 '不然吃這1塊就等於1份便當的熱量',
 '1.高熱量密度、低營養素',
 '蛋白質熱量只有26大卡左右',
 '其餘熱量約105大卡',
 '營養師有樺要說、經營臉書粉絲團',
 '（圖／營養師簡鈺樺提供）記者曹悅華／採訪報導吃GODIVA根本像在喝油',
 '若想將這個350大卡的熱量做個消耗平衡',
 '簡鈺樺營養師',
 '（圖／ETtoday新聞雲資料照）聽完營養師的分析',
 '▲從營養標示中會發現']

In [186]:
make_key_word_res(get_more_key_word_by_ner(content_list[4]))

{'j': ['三角戀'],
 'j+n': [],
 'nr': ['朱醫師', '朱育瑩', '李佳蓉', '朱', '症候群'],
 'ns': [],
 'nt': [],
 'nx': ['Pixabay',
  'C',
  'AcquiredImmuneDeficiencySyndrome',
  'C肝',
  'AIDS',
  'Youtube'],
 'nz': ['愛滋', 'ETtoday']}